In [2]:
import pandas as pd

df = pd.read_csv("data/vgsales.csv")
df = df.dropna(subset = ["Year"])
df['Year'] = df["Year"].astype("int")
df['Publisher'] = df['Publisher'].fillna("-")

df.info()
df.to_csv("data/vgsales_updated.csv")

<class 'pandas.core.frame.DataFrame'>
Index: 16327 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16327 non-null  int64  
 1   Name          16327 non-null  object 
 2   Platform      16327 non-null  object 
 3   Year          16327 non-null  int64  
 4   Genre         16327 non-null  object 
 5   Publisher     16327 non-null  object 
 6   NA_Sales      16327 non-null  float64
 7   EU_Sales      16327 non-null  float64
 8   JP_Sales      16327 non-null  float64
 9   Other_Sales   16327 non-null  float64
 10  Global_Sales  16327 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 1.5+ MB


In [3]:
#самые продаваемые игры по годам
game_year_sales = df.groupby(by = ["Name", "Year"])['Global_Sales'].sum().reset_index()
top_game = game_year_sales.groupby(by = ('Year'))["Global_Sales"].idxmax()

top_selling_game = game_year_sales.loc[top_game].reset_index().drop("index", axis =1)
top_selling_game.sort_values(by = "Year")

top_selling_game.to_csv("data/top_selling_games.csv")
top_selling_game.head()


,Name,Year,Global_Sales
0,Asteroids,1980,4.31
1,Pitfall!,1981,4.50
2,Pac-Man,1982,7.81
3,Baseball,1983,3.20
4,Duck Hunt,1984,28.31


In [4]:
#самые успешные издатели
#общие продажи издателей
publisher_sales = df.groupby(by="Publisher", as_index = False)["Global_Sales"].sum()
publisher_sales = publisher_sales.sort_values(by = "Global_Sales", ascending = False)

top_10_publishers = publisher_sales.iloc[0:10, :]

top_10_publishers.to_csv("data/top_10_publishers.csv")
top_10_publishers.head(10)

,Publisher,Global_Sales
360,Nintendo,1784.43
139,Electronic Arts,1093.39
22,Activision,721.41
456,Sony Computer Entertainment,607.28
525,Ubisoft,473.54
494,Take-Two Interactive,399.30
488,THQ,340.44
276,Konami Digital Entertainment,278.56
446,Sega,270.70
348,Namco Bandai Games,253.65


In [5]:
#топ 10 игр (по регионам)
region_sales = ['NA_Sales', "JP_Sales", 'EU_Sales', 'Other_Sales']
sales_by_region = df.groupby('Name', as_index=False)[region_sales].sum()
sales_by_region = sales_by_region.reset_index().drop("index", axis = 1)

top_games_by_region = pd.DataFrame()
for region in region_sales:
    #топ 10 в регионе
    top_10 = df.sort_values(by=region, ascending=False).head(10)
    top_10["Region"] = region.replace("_Sales", "")
    
    #переименование столбца продаж в общий формат
    top_10 = top_10.rename(columns = {region : "Sales"})
    
    #оставляем нужные колонки
    cols = ['Name', 'Platform', 'Genre', 'Publisher', 'Region', 'Sales']
    top_10 = top_10[cols]
    
    top_games_by_region = pd.concat([top_games_by_region, top_10])
    
top_games_by_region.to_csv("data/top_games_by_region.csv")
top_games_by_region.head(15)

,Name,Platform,Genre,Publisher,Region,Sales
0,Wii Sports,Wii,Sports,Nintendo,NA,41.49
1,Super Mario Bros.,NES,Platform,Nintendo,NA,29.08
9,Duck Hunt,NES,Shooter,Nintendo,NA,26.93
5,Tetris,GB,Puzzle,Nintendo,NA,23.20
2,Mario Kart Wii,Wii,Racing,Nintendo,NA,15.85
3,Wii Sports Resort,Wii,Sports,Nintendo,NA,15.75
15,Kinect Adventures!,X360,Misc,Microsoft Game Studios,NA,14.97
8,New Super Mario Bros. Wii,Wii,Platform,Nintendo,NA,14.59
7,Wii Play,Wii,Misc,Nintendo,NA,14.03
18,Super Mario World,SNES,Platform,Nintendo,NA,12.78


In [6]:
#количество проданных игр по платформам
platforms = df.groupby("Platform")["Global_Sales"].sum().reset_index()
platforms = platforms.sort_values(by = "Global_Sales", ascending = False).reset_index().drop('index', axis = 1)

platforms.to_csv("data/sales_by_platform.csv")
platforms.head()

,Platform,Global_Sales
0,PS2,1233.46
1,X360,969.61
2,PS3,949.35
3,Wii,909.81
4,DS,818.96


In [7]:
#топ 10 самых продаваемых игр по жанрам за всё время
#получение списка жанров
genres = df["Genre"].unique()
top_10_games_in_genre = pd.DataFrame()

for genre in genres:
    #фильтрация по каждому жанру
    df_genre = df[df["Genre"] == genre]
    
    #группировка по названию, суммирование продаж
    df_grouped = df_genre.groupby("Name", as_index = False)
    df_grouped = df_grouped.agg({"Global_Sales" : "sum",
                             "Platform" : "first",
                             "Publisher" : "first"})
    
    top_10 = df_grouped.sort_values(by = "Global_Sales", ascending = False).head(10)
    top_10["Genre"] = genre
    
    top_10_games_in_genre = pd.concat([top_10_games_in_genre, top_10], ignore_index = True)

#сохранение
top_10_games_in_genre.to_csv("data/top_10_games_in_genre.csv")
top_10_games_in_genre.head(15)

,Name,Global_Sales,Platform,Publisher,Genre
0,Wii Sports,82.74,Wii,Nintendo,Sports
1,Wii Sports Resort,33.00,Wii,Nintendo,Sports
2,Wii Fit,22.72,Wii,Nintendo,Sports
3,Wii Fit Plus,22.00,Wii,Nintendo,Sports
4,FIFA 15,19.02,PS4,Electronic Arts,Sports
5,FIFA 14,17.14,PS3,Electronic Arts,Sports
6,FIFA 16,16.44,PS4,Electronic Arts,Sports
7,Mario & Sonic at the Olympic Games,13.20,Wii,Sega,Sports
8,FIFA 12,13.15,PS3,Electronic Arts,Sports
9,FIFA Soccer 11,12.56,PS3,Electronic Arts,Sports


In [9]:
#по году и жанру -> сумма продаж
genre_year_sales = df.groupby(by = ['Year', 'Genre'], as_index = False)['Global_Sales'].sum()

#преобразование в матрицу жанр * год 
genre_year_pivot = genre_year_sales.pivot(index = "Genre", columns = "Year", values = "Global_Sales")

#если значение пустое
genre_year_pivot = genre_year_pivot.fillna(0)

genre_year_pivot.to_csv("data/genre_year_pivot.csv")
genre_year_pivot.head()

Year,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2020
Genre,,,,,,,,,,,,,,,,,,,,,
Action,0.34,14.84,6.52,2.86,1.85,3.52,13.74,1.12,1.75,4.64,...,139.36,117.64,118.96,122.04,125.22,99.02,70.70,19.91,0.01,0.0
Adventure,0.00,0.00,0.00,0.40,0.00,0.00,0.00,4.38,0.00,0.00,...,20.68,16.57,15.98,5.99,6.61,6.06,8.03,1.81,0.00,0.0
Fighting,0.77,0.00,0.00,0.00,0.00,1.05,0.00,5.42,0.00,0.00,...,32.15,14.89,22.68,9.51,7.21,16.15,7.78,3.86,0.00,0.0
Misc,2.71,0.00,0.87,2.14,1.45,0.00,0.00,0.00,0.00,1.28,...,76.94,96.86,56.08,22.92,25.65,23.68,11.69,1.17,0.00,0.0
Platform,0.00,6.93,5.03,6.93,0.69,43.17,9.39,1.74,27.73,20.66,...,41.09,31.90,28.11,18.55,25.12,8.89,6.05,2.07,0.00,0.0
